This notebook selects the morning and evening rush hour rides involving the Loop, and saves these in 4 separate JSON files.

In [1]:
import pandas as pd
import re

In [2]:
#what are the columns?
for chunk in pd.read_csv('Data/Taxi_Trips.csv',
                        iterator=True, chunksize=1,
                        low_memory=False):
    df = chunk
    print(df.columns)        
    break

Index(['Trip ID', 'Taxi ID', 'Trip Start Timestamp', 'Trip End Timestamp',
       'Trip Seconds', 'Trip Miles', 'Pickup Census Tract',
       'Dropoff Census Tract', 'Pickup Community Area',
       'Dropoff Community Area', 'Fare', 'Tips', 'Tolls', 'Extras',
       'Trip Total', 'Payment Type', 'Company', 'Pickup Centroid Latitude',
       'Pickup Centroid Longitude', 'Pickup Centroid Location',
       'Dropoff Centroid Latitude', 'Dropoff Centroid Longitude',
       'Dropoff Centroid  Location', 'Community Areas'],
      dtype='object')


In [3]:
df

,Trip ID,Taxi ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,...,Trip Total,Payment Type,Company,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location,Community Areas
0,faff2101f11c6cb018380f378e09a373aeec05ce,d527523310158da37c4d91bb68651d1e7b3c43fa9e8999...,01/12/2016 11:00:00 AM,01/12/2016 11:15:00 AM,960,0.3,NaN,NaN,8,6,...,$19.50,Credit Card,Taxi Affiliation Services,41.899602,-87.633308,POINT (-87.6333080367 41.899602111),41.944227,-87.655998,POINT (-87.6559981815 41.9442266014),37


First collect morning rush data.

After picking out the correct times, make the columns easier to deal with. Specifically,

1. Parse 'Trip Start Timestamp' and 'Trip End Timestamp' as datetime objects.
2. Read off latitude and longitude from 'Pickup Centroid Location' and 'Dropoff Centroid  Location'.
3. Parse 'Fare' as floats.

In [8]:
#Just the columns I need
columns = ['Trip Start Timestamp', 'Trip End Timestamp', 'Pickup Community Area',
           'Dropoff Community Area', 'Pickup Census Tract', 'Dropoff Census Tract', 'Fare']
new_columns = ['start_time', 'end_time', 'Pickup Census Tract', 'Dropoff Census Tract', 'fare']

#Write time parser
time_parser = lambda x: pd.datetime.strptime(x, '%m/%d/%Y %I:%M:%S %p')

#Parse Location to floats longitude and latitude
def location_parser(location):
    if re.match('POINT \(-\d*\.\d* \d*\.\d*\)', location):
        long, lat = re.findall('POINT \((-\d*\.\d*) (\d*\.\d*)\)', location)[0]
        return float(long), float(lat)
    else:
        return 0., 0.

#Parse Fare as float
def fare_parser(fare):
    """Keep nan's as is, but turn string '$_.__' into float _.__"""
    try: return float(fare[1:])
    except TypeError: return fare

df_am_from_loop_list = []
df_am_to_loop_list = []

count = 0
for chunk in pd.read_csv('Data/Taxi_Trips.csv',
                        iterator=True, chunksize=100000,
                        low_memory=False):
    count += 1
    
    #select trips from loop. there are about 2800 per chunk
    df1 = chunk[(chunk['Pickup Community Area'] == 32.)][columns].dropna(axis=0, how='any')
    #select trips from loop. there are about 4400 per chunk
    df2 = chunk[(chunk['Dropoff Community Area'] == 32.)][columns].dropna(axis=0, how='any')
    
    #pick hours
    df1['hour'] = df1['Trip Start Timestamp'].apply(lambda x: time_parser(x).hour) #create hour column
    df2['hour'] = df2['Trip Start Timestamp'].apply(lambda x: time_parser(x).hour) #create hour column

    #only keep morning rush rows
    df1 = df1[(df1.hour >= 8) & (df1.hour <= 10)]
    df2 = df2[(df2.hour >= 8) & (df2.hour <= 10)]
        
    if len(df1) > 0:
        #parse time
        df1['start_time'] = df1['Trip Start Timestamp'].apply(time_parser)
        df1['end_time'] = df1['Trip End Timestamp'].apply(time_parser)
        #parse fare
        df1['fare'] = df1['Fare'].apply(fare_parser)
        #append
        df_am_from_loop_list.append(df1[new_columns])
        
    if len(df2) > 0:
        #do the same
        df2['start_time'] = df2['Trip Start Timestamp'].apply(time_parser)
        df2['end_time'] = df2['Trip End Timestamp'].apply(time_parser)
        df2['fare'] = df2['Fare'].apply(fare_parser)
        df_am_to_loop_list.append(df2[new_columns]) #only append morning rush hour trips    
        
    #To keep track of progress, 
    #print the number of DataFrames appended so far, and length of the last one appended
    print(count, 
          len(df_am_from_loop_list[-1]),
          len(df_am_to_loop_list[-1]))

1 2589 4198
2 2538 4006
3 2574 3969
4 2607 4025
5 2676 3975
6 2577 3993
7 2582 3933
8 2592 4138
9 2509 3970
10 2524 3951
11 2533 3891
12 2582 4085
13 2576 4059
14 2561 3998
15 2539 4066
16 2572 4130
17 2502 3993
18 2605 4166
19 1992 2898
20 1385 1791
21 1318 1599
22 1359 1728
23 1261 1622
24 1261 1622
25 1261 1622
26 1261 1622
27 1261 1622
28 1261 1622
29 1261 1622
30 1261 1622
31 1261 1622
32 1261 1622
33 1261 1622
34 1261 1622
35 2 3
36 1 2
37 1 2
38 1 2
39 1 2
40 1 1
41 1 1
42 1 1
43 15 18
44 1426 1916
45 2714 3713
46 2865 3765
47 2851 3832
48 2814 3782
49 2889 3783
50 2910 3808
51 2831 3820
52 2923 3858
53 2909 3765
54 2787 3683
55 2838 3806
56 3042 3789
57 2785 3647
58 2928 3791
59 2883 3832
60 2883 3768
61 2781 3650
62 2935 3860
63 2788 3818
64 1678 2246
65 1678 2246
66 1678 2246
67 1678 2246
68 1678 2246
69 1678 2246
70 1678 2246
71 1678 2246
72 1678 2246
73 1678 2246
74 1678 2246
75 1678 2246
76 1678 2246
77 1678 2246
78 1678 2246
79 1678 2246
80 1678 2246
81 1678 2246
82 1678 

648 2562 4029
649 2544 3997
650 2547 3927
651 2601 4075
652 2550 4011
653 2486 4035
654 2564 3929
655 2564 3941
656 2624 4010
657 2527 4105
658 2512 3962
659 2499 4042
660 2539 4027
661 2496 3848
662 2518 3979
663 2490 3949
664 2592 4146
665 2522 3996
666 2656 4060
667 2526 3978
668 2498 3967
669 2586 4045
670 2590 4090
671 2554 4034
672 2606 4017
673 2530 4018
674 2566 4048
675 2555 4014
676 2500 3982
677 2547 4079
678 2512 4092
679 2437 3990
680 2548 4076
681 2569 4036
682 2501 4007
683 2584 3905
684 2451 4088
685 2521 4011
686 2579 4020
687 2559 3999
688 2497 4084
689 2612 3971
690 2478 3962
691 2494 4082
692 2501 3953
693 2602 4046
694 2573 3974
695 2566 4010
696 2581 3917
697 2543 4010
698 2536 4020
699 2536 4020
700 2531 4197
701 2445 3850
702 2545 3896
703 2587 4132
704 2503 4007
705 2569 4003
706 2580 4067
707 2464 3995
708 2574 4139
709 2580 3961
710 2529 4095
711 2540 4045
712 2501 4028
713 2541 3888
714 2599 3982
715 2585 4146
716 2503 3918
717 2587 4015
718 2497 4000
719 26

In [17]:
sum(len(chunk) for chunk in df_am_from_loop_list)

2468462

In [18]:
sum(len(chunk) for chunk in df_am_to_loop_list)

3878407

In [11]:
#concatenate to make full DataFrames
df_am_from_loop = pd.concat(df_am_from_loop_list)
df_am_to_loop = pd.concat(df_am_to_loop_list)

In [12]:
#save the morning taxi rides to and from the loop as a new JSON file
df_am_from_loop.to_json('Data/am_from_loop.json')
df_am_to_loop.to_json('Data/am_to_loop.json')

OK, there are segments in the data that are just not filled out. Any patterns to this? Make sure it's not something that introduces a bias.

Next do evening rush.

In [13]:
df_pm_from_loop_list = []
df_pm_to_loop_list = []

#Do small scale thing first
for chunk in pd.read_csv('Data/Taxi_Trips.csv',
                        iterator=True, chunksize=100000,
                        low_memory=False):
    
    #select trips from loop. there are about 8200 per chunk
    df1 = chunk[(chunk['Pickup Community Area'] == 32.)][columns].dropna(axis=0, how='any')
    #select trips from loop. there are about 4900 per chunk
    df2 = chunk[(chunk['Dropoff Community Area'] == 32.)][columns].dropna(axis=0, how='any')
    
    #pick hours
    df1['hour'] = df1['Trip Start Timestamp'].apply(lambda x: time_parser(x).hour) #create hour column
    df2['hour'] = df2['Trip Start Timestamp'].apply(lambda x: time_parser(x).hour) #create hour column
    
    #only keep morning rush rows
    df1 = df1[(df1.hour >= 16) & (df1.hour <= 20)]
    df2 = df2[(df2.hour >= 16) & (df2.hour <= 20)]
    
    if len(df1) > 0:
        #parse time
        df1['start_time'] = df1['Trip Start Timestamp'].apply(time_parser)
        df1['end_time'] = df1['Trip End Timestamp'].apply(time_parser)
        #parse fare
        df1['fare'] = df1['Fare'].apply(fare_parser)
        #append
        df_pm_from_loop_list.append(df1[new_columns])
        
    if len(df2) > 0:
        #do the same
        df2['start_time'] = df2['Trip Start Timestamp'].apply(time_parser)
        df2['end_time'] = df2['Trip End Timestamp'].apply(time_parser)
        df2['fare'] = df2['Fare'].apply(fare_parser)
        df_pm_to_loop_list.append(df2[new_columns]) #only append morning rush hour trips    
    
    #To keep track of progress, 
    #print the number of DataFrames appended so far, and length of the last one appended
    print(len(df_pm_from_loop_list), 
          len(df_pm_from_loop_list[-1]),
          len(df_pm_to_loop_list[-1]))

1 7315 4639
2 7326 4748
3 7361 4700
4 7310 4604
5 7311 4624
6 7368 4687
7 7403 4804
8 7277 4730
9 7274 4758
10 7492 4964
11 7372 4742
12 7539 4775
13 7348 4715
14 7511 4823
15 7304 4702
16 7365 4682
17 7128 4751
18 7327 4873
19 5617 3611
20 3619 2279
21 3552 2131
22 3612 2247
23 3363 1974
23 3363 1974
23 3363 1974
23 3363 1
24 1 1
24 1 1
24 1 1
24 1 1
24 1 1
24 1 1
24 1 1
24 1 1
25 2 4
26 1 1
27 1 1
28 6 6
29 1 6
30 2 3
31 3 2
32 1 2
33 32 33
34 4212 2860
35 8182 5579
36 8153 5616
37 8152 5583
38 8257 5607
39 8248 5653
40 8076 5604
41 8108 5512
42 8216 5624
43 8077 5455
44 8120 5605
45 8366 5671
46 8139 5513
47 8135 5739
48 8146 5635
49 8123 5611
50 8308 5657
51 8285 5540
52 8250 5700
53 8153 5571
54 4718 3152
54 4718 3152
54 4718 3152
54 4718 3152
54 4718 3152
54 4718 3152
54 4718 3152
54 4718 3152
54 4718 3152
54 4718 3152
54 4718 3152
54 4718 3152
54 4718 3152
54 4718 3152
54 4718 3152
54 4718 3152
54 4718 1
54 4718 1
54 4718 1
54 4718 1
54 4718 1
54 4718 1
55 1 1
55 1 1
55 1 1
56 1

530 7390 4762
531 7468 4739
532 7122 4570
533 7290 4574
534 7366 4609
535 7379 4696
536 7433 4737
537 7199 4778
538 7367 4796
539 7377 4661
540 7469 4881
541 7331 4700
542 7281 4643
543 7388 4718
544 7399 4749
545 7265 4711
546 7396 4812
547 7303 4794
548 7395 4794
549 7262 4763
550 7423 4693
551 7238 4697
552 7344 4630
553 7384 4827
554 7262 4720
555 7243 4675
556 7541 4710
557 7439 4748
558 7400 4855
559 7224 4671
560 7393 4636
561 7379 4663
562 7339 4697
563 7309 4729
564 7350 4841
565 7319 4732
566 7403 4783
567 7439 4824
568 7313 4730
569 7497 4705
570 7360 4722
571 7371 4678
572 7360 4754
573 7216 4789
574 7300 4681
575 7293 4847
576 7320 4824
577 7399 4667
578 7348 4760
579 7354 4654
580 7431 4691
581 7293 4760
582 7448 4666
583 7231 4765
584 7338 4847
585 7436 4767
586 7428 4785
587 7288 4724
588 7280 4719
589 7329 4720
590 7311 4775
591 7176 4734
592 7319 4656
593 7352 4622
594 7404 4763
595 7301 4785
596 7347 4675
597 7364 4652
598 7423 4709
599 7357 4608
600 7293 4737
601 73

In [14]:
#concatenate to make full DataFrames
df_pm_from_loop = pd.concat(df_pm_from_loop_list)
df_pm_to_loop = pd.concat(df_pm_to_loop_list)

In [16]:
#save the morning taxi rides to and from the loop as a new JSON file
df_pm_from_loop.to_json('Data/pm_from_loop.json')
df_pm_to_loop.to_json('Data/pm_to_loop.json')

## Old code
I realized that the centroid locations are simply tied to census tracts.

In [48]:
#Just the columns I need
columns = ['Trip Start Timestamp', 'Trip End Timestamp', 'Pickup Community Area',
           'Dropoff Community Area', 'Pickup Centroid Location', 'Dropoff Centroid  Location', 'Fare']
new_columns = ['start_time', 'end_time', 'Pickup Community Area',
              'Dropoff Community Area', 'pickup_long', 'pickup_lat',
              'dropoff_long', 'dropoff_lat']

#Write time parser
time_parser = lambda x: pd.datetime.strptime(x, '%m/%d/%Y %I:%M:%S %p')

#Parse Location to floats longitude and latitude
def location_parser(location):
    if re.match('POINT \(-\d*\.\d* \d*\.\d*\)', location):
        long, lat = re.findall('POINT \((-\d*\.\d*) (\d*\.\d*)\)', location)[0]
        return float(long), float(lat)
    else:
        return 0., 0.

#Parse Fare as float
def fare_parser(fare):
    """Keep nan's as is, but turn string '$_.__' into float _.__"""
    try: return float(fare[1:])
    except TypeError: return fare

df_am_from_loop_list = []
df_am_to_loop_list = []

count = 0
for chunk in pd.read_csv('Data/Taxi_Trips.csv',
                        iterator=True, chunksize=100000,
                        low_memory=False):
    count += 1
    
    #select trips from loop. there are about 2800 per chunk
    df1 = chunk[(chunk['Pickup Community Area'] == 32.)][columns].dropna(axis=0, how='any')
    #select trips from loop. there are about 4400 per chunk
    df2 = chunk[(chunk['Dropoff Community Area'] == 32.)][columns].dropna(axis=0, how='any')
    
    #pick hours
    df1['hour'] = df1['Trip Start Timestamp'].apply(lambda x: time_parser(x).hour) #create hour column
    df2['hour'] = df2['Trip Start Timestamp'].apply(lambda x: time_parser(x).hour) #create hour column

    #only keep morning rush rows
    df1 = df1[(df1.hour >= 8) & (df1.hour <= 10)]
    df2 = df2[(df2.hour >= 8) & (df2.hour <= 10)]
        
    if len(df1) > 0:
        #parse time
        df1['start_time'] = df1['Trip Start Timestamp'].apply(time_parser)
        df1['end_time'] = df1['Trip End Timestamp'].apply(time_parser)
        #read off longitude, latitude. there's an error when dfs are empty.
        df1[['pickup_long', 'pickup_lat']] = df1['Pickup Centroid Location'].apply(location_parser).apply(pd.Series)
        df1[['dropoff_long', 'dropoff_lat']] = df1['Dropoff Centroid  Location'].apply(location_parser).apply(pd.Series)
        #parse fare
        df1['fare'] = df1['Fare'].apply(fare_parser)
        #append
        df_am_from_loop_list.append(df1[new_columns])
        
    if len(df2) > 0:
        #do the same
        df2['start_time'] = df2['Trip Start Timestamp'].apply(time_parser)
        df2['end_time'] = df2['Trip End Timestamp'].apply(time_parser)
        df2[['pickup_long', 'pickup_lat']] = df2['Pickup Centroid Location'].apply(location_parser).apply(pd.Series)
        df2[['dropoff_long', 'dropoff_lat']] = df2['Dropoff Centroid  Location'].apply(location_parser).apply(pd.Series)
        df2['fare'] = df2['Fare'].apply(fare_parser)
        df_am_to_loop_list.append(df2[new_columns]) #only append morning rush hour trips    
    
    #To keep track of progress, 
    #print the number of DataFrames appended so far, and length of the last one appended
    print(count, 
          len(df_am_from_loop_list[-1]),
          len(df_am_to_loop_list[-1]))

1 2801 4590
2 2739 4418
3 2771 4359
4 2832 4466
5 2880 4372
6 2782 4365
7 2806 4331
8 2797 4535
9 2699 4347
10 2726 4307
11 2732 4274
12 2794 4461
13 2776 4477
14 2772 4362
15 2716 4442
16 2771 4505
17 2716 4392
18 2790 4548
19 2099 3105
20 1426 1833
21 1363 1644
22 1396 1768
23 1305 1655
24 1305 1655
25 1305 1655
26 1305 1655
27 1305 1655
28 1305 1
29 1305 1
30 1305 1
31 1305 1
32 1305 1
33 1305 1
34 1305 1
35 7 8
36 2 2
37 1 2
38 1 2
39 1 2
40 1 1
41 1 1
42 1 1
43 30 30
44 1600 2142
45 3020 4156
46 3192 4257
47 3148 4277
48 3145 4227
49 3163 4251
50 3223 4260
51 3158 4271
52 3257 4347
53 3199 4223
54 3096 4166
55 3148 4255
56 3337 4257
57 3103 4116
58 3263 4213
59 3205 4280
60 3219 4201
61 3080 4121
62 3226 4315
63 3136 4293
64 1874 2503
65 1874 2503
66 1874 2503
67 1874 2503
68 1874 2503
69 1874 2503
70 1874 2503
71 1874 2503
72 1874 2503
73 1874 2503
74 1874 2503
75 1874 2503
76 1874 2503
77 1874 2503
78 1874 1
79 1874 1
80 1874 1
81 1874 1
82 1874 1
83 1874 1
84 1874 1
85 1 1
86 1

651 2791 4462
652 2769 4389
653 2670 4420
654 2792 4287
655 2754 4293
656 2819 4392
657 2713 4498
658 2708 4365
659 2687 4458
660 2734 4386
661 2706 4255
662 2716 4330
663 2689 4341
664 2777 4518
665 2722 4380
666 2836 4443
667 2718 4339
668 2716 4352
669 2790 4436
670 2815 4461
671 2759 4402
672 2788 4409
673 2747 4390
674 2747 4451
675 2765 4402
676 2700 4369
677 2741 4453
678 2729 4456
679 2633 4351
680 2746 4479
681 2749 4397
682 2722 4366
683 2777 4286
684 2659 4448
685 2710 4389
686 2788 4401
687 2791 4385
688 2696 4462
689 2808 4360
690 2683 4336
691 2703 4497
692 2723 4327
693 2774 4412
694 2782 4321
695 2778 4419
696 2783 4290
697 2733 4400
698 2732 4422
699 2736 4423
700 2721 4584
701 2639 4251
702 2766 4290
703 2795 4518
704 2674 4381
705 2783 4383
706 2757 4443
707 2634 4381
708 2753 4545
709 2787 4341
710 2720 4455
711 2746 4415
712 2707 4430
713 2724 4288
714 2806 4393
715 2776 4549
716 2695 4320
717 2787 4382
718 2694 4354
719 2847 4365
720 2658 4407
721 2701 4303
722 27

In [60]:
df_pm_from_loop_list = []
df_pm_to_loop_list = []

#Do small scale thing first
for chunk in pd.read_csv('Data/Taxi_Trips.csv',
                        iterator=True, chunksize=100000,
                        low_memory=False):
    
    #select trips from loop. there are about 8200 per chunk
    df1 = chunk[(chunk['Pickup Community Area'] == 32.)][columns].dropna(axis=0, how='any')
    #select trips from loop. there are about 4900 per chunk
    df2 = chunk[(chunk['Dropoff Community Area'] == 32.)][columns].dropna(axis=0, how='any')
    
    #pick hours
    df1['hour'] = df1['Trip Start Timestamp'].apply(lambda x: time_parser(x).hour) #create hour column
    df2['hour'] = df2['Trip Start Timestamp'].apply(lambda x: time_parser(x).hour) #create hour column
    
    #only keep morning rush rows
    df1 = df1[(df1.hour >= 16) & (df1.hour <= 20)]
    df2 = df2[(df2.hour >= 16) & (df2.hour <= 20)]
    
    if len(df1) > 0:
        #parse time
        df1['start_time'] = df1['Trip Start Timestamp'].apply(time_parser)
        df1['end_time'] = df1['Trip End Timestamp'].apply(time_parser)
        #read off longitude, latitude. there's an error when dfs are empty.
        df1[['pickup_long', 'pickup_lat']] = df1['Pickup Centroid Location'].apply(location_parser).apply(pd.Series)
        df1[['dropoff_long', 'dropoff_lat']] = df1['Dropoff Centroid  Location'].apply(location_parser).apply(pd.Series)
        #parse fare
        df1['fare'] = df1['Fare'].apply(fare_parser)
        #append
        df_pm_from_loop_list.append(df1[new_columns])
        
    if len(df2) > 0:
        #do the same
        df2['start_time'] = df2['Trip Start Timestamp'].apply(time_parser)
        df2['end_time'] = df2['Trip End Timestamp'].apply(time_parser)
        df2[['pickup_long', 'pickup_lat']] = df2['Pickup Centroid Location'].apply(location_parser).apply(pd.Series)
        df2[['dropoff_long', 'dropoff_lat']] = df2['Dropoff Centroid  Location'].apply(location_parser).apply(pd.Series)
        df2['fare'] = df2['Fare'].apply(fare_parser)
        df_pm_to_loop_list.append(df2[new_columns]) #only append morning rush hour trips    
    
    #To keep track of progress, 
    #print the number of DataFrames appended so far, and length of the last one appended
    print(len(df_pm_from_loop_list), 
          len(df_pm_from_loop_list[-1]),
          len(df_pm_to_loop_list[-1]))

1 8138 4835
2 8216 4938
3 8254 4905
4 8175 4820
5 8092 4822
6 8137 4881
7 8283 4990
8 8083 4920
9 8159 4972
10 8289 5179
11 8244 4933
12 8370 4985
13 8117 4931
14 8315 5011
15 8121 4905
16 8200 4894
17 7980 4962
18 8227 5078
19 6093 3732
20 3799 2313
21 3736 2160
22 3795 2278
23 3534 2010
23 3534 1
23 3534 1
23 3534 1
24 1 1
24 1 1
24 1 1
24 1 1
24 1 1
24 1 1
24 1 1
24 1 1
25 10 6
26 2 2
27 1 2
28 8 6
29 4 1
30 3 4
31 4 3
32 1 3
33 96 44
34 4886 3013
35 9491 5894
36 9503 5877
37 9441 5876
38 9520 5901
39 9593 5933
40 9439 5877
41 9360 5783
42 9489 5924
43 9443 5737
44 9370 5897
45 9635 5940
46 9407 5785
47 9480 6007
48 9419 5920
49 9494 5905
50 9705 5952
51 9637 5861
52 9587 5998
53 9509 5863
54 5446 3321
54 5446 3321
54 5446 3321
54 5446 3321
54 5446 3321
54 5446 3321
54 5446 3321
54 5446 3321
54 5446 3321
54 5446 1
54 5446 1
54 5446 1
54 5446 1
54 5446 1
54 5446 1
54 5446 1
54 5446 1
54 5446 1
54 5446 1
54 5446 1
54 5446 1
54 5446 1
55 1 1
55 1 1
55 1 1
56 1 1
56 1 1
56 1 1
56 1 1
56

539 8293 4950
540 7958 4773
541 8138 4786
542 8199 4833
543 8249 4893
544 8273 4944
545 8005 4973
546 8234 4996
547 8209 4871
548 8295 5078
549 8146 4908
550 8134 4830
551 8239 4889
552 8285 4953
553 8034 4916
554 8230 5009
555 8154 5010
556 8191 4998
557 8129 4939
558 8278 4903
559 8063 4904
560 8217 4794
561 8257 5042
562 8145 4944
563 8130 4876
564 8377 4912
565 8280 4948
566 8295 5065
567 8103 4892
568 8250 4857
569 8210 4887
570 8188 4902
571 8106 4945
572 8182 5045
573 8138 4934
574 8238 4990
575 8294 5030
576 8134 4922
577 8362 4892
578 8222 4909
579 8289 4897
580 8204 4956
581 8053 4987
582 8156 4886
583 8179 5044
584 8160 5026
585 8305 4889
586 8182 4965
587 8154 4856
588 8328 4903
589 8093 4971
590 8324 4890
591 8032 4935
592 8177 5063
593 8296 4949
594 8264 4986
595 8137 4903
596 8164 4925
597 8194 4908
598 8165 4965
599 8032 4929
600 8159 4856
601 8167 4821
602 8196 4965
603 8097 4983
604 8174 4880
605 8216 4830
606 8313 4912
607 8159 4790
608 8076 4928
609 8166 4906
610 83